In [1]:
import emails
import hme
import sre
import uts
import re
import csv
import statistics

In [4]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.chunk import ne_chunk

[nltk_data] Downloading package punkt to /home/bitnami/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
emailstext = emails.getAll("allfilenames.txt")

In [7]:
def bmstats(bmpgl_emailstext):
    blankpgs = []
    noemails = []
    yesemails = []
    sa = []
    listbms = re.split("BOOKMARKend---",bmpgl_emailstext)
    for bm in listbms:
        listTS = uts.getTS(bm)
        listSo = sre.findRawSenders_pg(bm)
        deqfile = re.findall("deq.*_X",bm)
        nowords = re.findall("^\n*\W*[\x0cdeq[_0-9b]+.txt---endPAGE]+---$",bm)
        if nowords != []: # no text
            for noword in nowords:
                sa.append(noword)
                noword = re.sub('\n','',noword)
                nochar = re.findall("^\s*\x0cdeq[_0-9b]+.txt$",noword)
                nolett = re.findall("^\W*\x0cdeq[_0-9b]+.txt$",noword)
                for nos in nochar: #no text
                    blankpgs.append(bm)
        elif nowords == []: #there is text
            if listTS == [] and listSo == []: #no email
                noemails.append(bm)
            if listTS != [] or listSo != []: #yes email
                yesemails.append(bm)
    return noemails

In [8]:
noe = bmstats(emailstext)

In [9]:
len(noe)

1253

In [63]:
import time
start_time = time.time()
listtokem = []
for att in noe:
    tokem =nltk.word_tokenize(att)
    listtokem.append(tokem)
print("%s seconds" % (time.time() - start_time))

19.262669801712036 seconds


In [822]:
hg=[]
for tok in (listtokem):
    g = len(tok)
    d = tok
    new = (g)
    hg.append(new)
hg.sort()
u = list(dict.fromkeys(hg))
u.sort()
#print(u) #list of lengths, to know cuttoffs

In [823]:
h=[]
for tok in (listtokem):
    g = len(tok)
    d = tok
    new = (d,g)
    h.append(new)
h.sort()

In [564]:
def setpairs(listtokem): #pairs every bm with each other
    dupspairs = []
    dups = []
    comparer = []
    xx = []
    for tokems in listtokem:
        listtokem1=[] 
        for dems in listtokem:
            if dems != tokems:
                listtokem1.append(dems) #new list with everything but given tokem
        for tokems1 in listtokem1:# compare tokem to every other tokem1
            if [tokems,tokems1] not in comparer: #and [tokems1,tokems] not in comparer:
                comparer.append([tokems,tokems1])
    xx.extend(comparer)
    return xx
def checklgt(xx): #if they have similar lengths (number of tokens), returns
    xxa = []
    no = []
    for pair in xx:
        p1 = pair[0]
        p2 = pair[1]
        tok1 = p1[0]
        tok2 = p2[0]
        if len(p1[0]) > 4 and len(p2[0])>4:
            deqf1 = tok1[-5]
            deqf2 = tok2[-5]
        if (p1[1])<(p2[1]):
            if (p1[1])/(p2[1]) > .8:
                xxa.append(pair)
            else:
                no.extend(pair)
                #pass
        elif (p1[1])>(p2[1]):
            if (p2[1])/(p1[1]) > .8:
                xxa.append(pair)
            else:
                no.extend(pair)
                #pass
        elif len(p1)==len(p2):
            xxa.append(pair)
    return xxa
def matchtoks(xxa): #if over 80% of tokens match, returns
    no=[]
    dups = []
    dupairfs = []
    for paira in xxa:
        match1 = []
        match2 = []
        pa1 = paira[0]
        pa2 = paira[1]
        tok1 = pa1[0]
        tok2 = pa2[0]
        if len(pa1[0]) > 4 and len(pa2[0]) >4:
            deqfa1 = tok1[-5]
            deqfa2 = tok2[-5]
        avg = (len(tok1)+len(tok2))/2
        for wd in tok1:
            if wd in tok2:
                match1.append(wd)
        for wd in tok2:
            if wd in tok1:
                match2.append(wd)
        score = (len(match1)+len(match2))/2
        if score/avg > .8:
            dupairf = [deqfa1,deqfa2]
            dupairfs.append(dupairf)
            dupair = [tuple(pa1),tuple(pa2)]
            dupspairs.append(dupair)
            dups.extend(paira)
        else:
            no.extend(paira)
            pass
    def fwdNbckwd(pairlists):
        reww = []
        for yus in pairlists:
            new = [yus[1],yus[0]]
            reww.append(new)
        newlist = list(reww + pairlists)
        return newlist
    dupairfsx2 = fwdNbckwd(dupairfs)
    return dupairfsx2

In [486]:
def combine(u):
        ndl = {}
        nts = []
        for lists in u:
            f = lists[0]
            s = lists[1]
            newdic = {}
            if f not in [*ndl]:
                newdic[f] = [s]
                ndl.update(newdic)
            else:
                ndl[f].append(s)
                pass
        lndl = tolist(ndl)
        return lndl
def combine1(u):
        ndl = {}
        nts = []
        for lists in u:
            f = lists[0]
            s = lists[1]
            newdic = {}
            if f not in [*ndl]:
                newdic[f] = s
                ndl.update(newdic)
            else:
                ndl[f].append(s)
                pass
        lndl = tolist(ndl)
        lndl.extend(nts)
        return lndl

In [659]:
def smoothout(x):
    def elimrepeats(medatt4):
        newu = []
        for xxxs in medatt4:
            xxxs.sort()
            du1d = list(dict.fromkeys(tuple(xxxs)))
            newu.append(tuple(du1d))
        len(newu)
        newud = list(dict.fromkeys(newu))
        newud.sort()
        len(newu)
        def orderbylen(elem):
            return len(elem)
        newud.sort(key=orderbylen)
        newud1 = emails.revlist(newud)
        newud1
        return newud1
    def consolidate(newud1):
        tu = []
        du = []
        for tuples in newud1:
            for deqs in tuples:
                if deqs not in du:
                    du.append(deqs)
                    tu.append(tuples)
                else:
                    pass
        tu1 = list(dict.fromkeys(tu))
        nu = []
        for xxxs in tu1:
            lxxxs = list(xxxs)
            lxxxs.sort()
            du1d = list(dict.fromkeys(tuple(lxxxs)))
            nu.append(tuple(du1d))
        nu1 = list(dict.fromkeys(nu))
        nu1.sort()
        return nu1
    def elimrepeats2(nu1):
        nu2 = []
        for nus in nu1:
            nu2.append(nus)
        yo = []
        for nus in nu1:
            for deqs in nus:
                for nups in nu1:
                    if nus != nups and deqs in nups:
                        new = list(nus + nups)
                        new1 = list(dict.fromkeys(new))
                        new1.sort()
                        yo.append(tuple(new1))

        yos = list(dict.fromkeys(yo))
        broo = []
        for tups in yos:
            for deqs in tups:
                for nups in nu1:
                    if deqs in nups and nups in nu2:
                        nu2.remove(nups)

        b = list(nu2 + yos)   
        bb = list(dict.fromkeys(b))
        bb.sort()
        #CHECK
        all1 = []
        for tups in bb:
            for deqs in tups:
                all1.append(deqs)
        all2 = list(dict.fromkeys(all1))
        if len(all1) == len(all2):
            return bb
        else:
            print("error")
    return elimrepeats2(consolidate(elimrepeats(x)))

In [490]:
def tolist(r):
        listr = []
        for key, value in r.items():
            v = []
            v.append(key)
            v.extend(value)
            listr.append(v)
        return listr

In [693]:
def begend(words):
    beg = words[0:100]
    end = words[-106:]
    begend = list(beg + end)
    return begend

In [731]:
short=[]
blank=[]
med=[]
medlong=[]
long=[]
sprlong=[]
rlylong=[]
rlyrlylong = []
for hs in h:
    if hs[1] == 5:
        blank.append(hs)
    elif hs[1] in range(6,13):
        short.append(hs)
    elif hs[1] in range(14,216):
        new = (begend(hs[0]),hs[1])
        med.append(hs)
    elif hs[1] in range(217,637):
        new = (begend(hs[0]),hs[1])
        medlong.append(new)
    elif hs[1] in range(638,1003):
        new = (begend(hs[0]),hs[1])
        long.append(new)
    elif hs[1] in range(1004,1687):
        new = (begend(hs[0]),hs[1])
        sprlong.append(new)
    elif hs[1] in range(1688,5108):
        new = (begend(hs[0]),hs[1])
        rlylong.append(new)
    elif hs[1] in range(5109,7871):
        new = (begend(hs[0]),hs[1])
        rlyrlylong.append(new)


In [737]:
start_time = time.time()
shatt1 = setpairs(short)
shatt2 = checklgt(shatt1)
shatt3 = matchtoks(shatt2)
shatt3.sort()
shatt4 = combine(shatt3)
shatt4.sort()
shatt5 = smoothout(shatt4)
print("%s seconds" % (time.time() - start_time))

0.10443806648254395 seconds


In [576]:
start_time = time.time()
medatt1 = setpairs(med)
medatt2 = checklgt(medatt1)
medatt3 = matchtoks(medatt2)
medatt3.sort()
medatt4 = combine(medatt3)
medatt4.sort()
medatt5 = smoothout(medatt4)
print("%s seconds" % (time.time() - start_time))

90.45188283920288 seconds


In [732]:
start_time = time.time()
medlatt1 = setpairs(medlong)
medlatt2 = checklgt(medlatt1)
medlatt3 = matchtoks(medlatt2)
medlatt3.sort()
medlatt4 = combine(medlatt3)
medlatt4.sort()
medlatt5 = smoothout(medlatt4)
print("%s seconds" % (time.time() - start_time))

500.3269476890564 seconds


In [733]:
start_time = time.time()
longatt1 = setpairs(long)
longatt2 = checklgt(longatt1)
longatt3 = matchtoks(longatt2)
longatt3.sort()
longatt4 = combine(longatt3)
longatt4.sort()
longatt5 = smoothout(longatt4)
print("%s seconds" % (time.time() - start_time))

76.94878458976746 seconds


In [734]:
start_time = time.time()
sprlongatt1 = setpairs(sprlong)
sprlongatt2 = checklgt(sprlongatt1)
sprlongatt3 = matchtoks(sprlongatt2)
sprlongatt3.sort()
sprlongatt4 = combine(sprlongatt3)
sprlongatt4.sort()
sprlongatt5 = smoothout(sprlongatt4)
print("%s seconds" % (time.time() - start_time))

23.68197274208069 seconds


In [735]:
start_time = time.time()
rlylongatt1 = setpairs(rlylong)
rlylongatt2 = checklgt(rlylongatt1)
rlylongatt3 = matchtoks(rlylongatt2)
rlylongatt3.sort()
rlylongatt4 = combine(rlylongatt3)
rlylongatt4.sort()
rlylongatt5 = smoothout(rlylongatt4)
print("%s seconds" % (time.time() - start_time))

18.112900733947754 seconds


In [736]:
start_time = time.time()
rlyrlylongatt1 = setpairs(rlyrlylong)
rlyrlylongatt2 = checklgt(rlyrlylongatt1)
rlyrlylongatt3 = matchtoks(rlyrlylongatt2)
rlyrlylongatt3.sort()
rlyrlylongatt4 = combine(rlyrlylongatt3)
rlyrlylongatt4.sort()
rlyrlylongatt5 = smoothout(rlyrlylongatt4)
print("%s seconds" % (time.time() - start_time))

0.38559794425964355 seconds


In [706]:
r = []
for tok in listtokem:
    if len(tok) >4:
        deq = tok[-5]
        r.append(deq)
len(r)

1252

In [788]:
alldups = list(shatt5+medatt5+medlatt5+longatt5+sprlongatt5+rlylongatt5+rlyrlylongatt5)
alldups_deqs = []
for tups in alldups:
    for deqs in tups:
        alldups_deqs.append(deqs)
len(alldups_deqs)
alldd=[]
for d in alldups_deqs:
    nd = re.findall("deq.*txt",d)
    alldd.extend(nd)
len(alldd)
len(list(dict.fromkeys(alldd)))
#len(alldd)

659

In [786]:
alldups[0:3]

[('deq24_b217_541_541_1.txt',
  'deq24_b396_980_980_1.txt',
  'deq24_b669_1724_1724_1.txt',
  'deq24_b291_700_700_1.txt',
  'deq24_b257_636_636_1.txt',
  'deq24_b230_574_574_1.txt',
  'deq24_b535_1368_1368_1.txt',
  'deq24_b228_571_571_1.txt',
  'deq24_b49_113_113_1.txt',
  'deq24_b155_393_393_1.txt',
  'deq24_b645_1675_1675_1.txt',
  'deq24_b477_1212_1212_1.txt',
  'deq24_b706_1888_1888_1.txt',
  'deq24_b356_915_915_1.txt',
  'deq24_b303_720_720_1.txt',
  'deq24_b312_733_733_1.txt',
  'deq24_b280_687_687_1.txt',
  'deq24_b339_880_880_1.txt',
  'deq24_b98_263_263_1.txt',
  'deq24_b310_731_731_1.txt',
  'deq24_b212_532_532_1.txt',
  'deq24_b653_1691_1691_1.txt',
  'deq24_b308_728_728_1.txt',
  'deq24_b646_1676_1676_1.txt',
  'deq24_b354_913_913_1.txt',
  'deq24_b156_394_394_1.txt',
  'deq24_b64_140_140_1.txt'),
 ('deq11_b147_866_866_1.txt', 'deq11_b96_211_211_1.txt'),
 ('deq11_b14_23_24_2.txt', 'deq14_b128_305_306_2.txt')]

In [814]:
allnond=[]
for d in r:
    nd = re.findall("deq.*txt",d)
    allnond.extend(nd)
alldups_deqs1 = []
for d in alldups_deqs:
    nd = re.findall("deq.*txt",d)
    alldups_deqs1.extend(nd)
alldups1 = []
for d in alldups:
    newd = []
    for ds in d:
        nd = re.findall("deq.*txt",ds)
        newd.extend(nd)
    newd.sort()
    alldups1.append(tuple(newd))
len(allnond)
notdups = []
for rs in allnond:
    if rs not in alldd:
        notdups.append(rs)
len(notdups)

593

In [818]:
def takeFir(elem):
    if type(elem) == str:
        return elem
    if type(elem) == tuple:
        return elem[0]

In [819]:
total = list(alldups1 + notdups)
total.sort(key=takeFir)

In [821]:
total[0:10]

['deq01_b24_1845_3570_1726.txt',
 'deq02_b299_3506_3506_1.txt',
 'deq04_b20_1412_1426_15.txt',
 'deq04_b270_3577_3579_3.txt',
 'deq04_b77_2967_3228_251.txt',
 ('deq11_b102_237_239_3.txt',
  'deq14_b242_925_928_4.txt',
  'deq14_b319_1065_1067_3.txt',
  'deq14_b326_1077_1080_4.txt',
  'deq14_b333_1095_1098_4.txt',
  'deq14_b338_1106_1109_4.txt',
  'deq14_b380_1260_1263_4.txt',
  'deq14_b382_1266_1268_3.txt',
  'deq15_b13_80_83_4.txt',
  'deq15_b35_117_120_4.txt',
  'deq15_b38_125_128_4.txt',
  'deq25_b59_753_755_3.txt'),
 ('deq11_b103_240_241_2.txt', 'deq24_b577_1492_1493_2.txt'),
 'deq11_b104_242_242_1.txt',
 'deq11_b105_243_243_1.txt',
 'deq11_b10_17_18_2.txt']